In [ ]:
%matplotlib inline
from __future__ import print_function, absolute_import

### 1) No-op

Step 1) Simple dircol traj. optimization.

Step 2) Simple supervised learning on (x, u) pairs, with no constraints on policy.

### 2) GPS

Step 1) dircol traj. optimization using Augmented Lagrangian

Step 2) Supervised learning on (x, u) pairs, using Augmented Lagrangian

### 3) Interactive Control

Step 1) dircol traj. optimization with L2 augmentation and L2 trajectory distance penalty.

Step 2) Simple supervised learning on (x, u) pairs, with L2 augmentation and L2 policy distance penalty.